In [1]:
import pandas as pd
import numpy as np

In [2]:
#read files
monday = pd.read_csv('monday.csv', sep=';', parse_dates=True, index_col=0)
tuesday = pd.read_csv('tuesday.csv', sep=';', parse_dates=True, index_col=0)
wednesday = pd.read_csv('wednesday.csv', sep=';',parse_dates=True, index_col=0)
thursday = pd.read_csv('thursday.csv', sep=';', parse_dates=True, index_col=0)
friday = pd.read_csv('friday.csv', sep=';', parse_dates=True, index_col=0)

In [3]:
monday

,customer_no,location
timestamp,,
2019-09-02 07:03:00,1,dairy
2019-09-02 07:03:00,2,dairy
2019-09-02 07:04:00,3,dairy
2019-09-02 07:04:00,4,dairy
2019-09-02 07:04:00,5,spices
...,...,...
2019-09-02 21:49:00,1442,checkout
2019-09-02 21:49:00,1444,checkout
2019-09-02 21:49:00,1445,dairy


In [4]:
tuesday

,customer_no,location
timestamp,,
2019-09-03 07:02:00,1,fruit
2019-09-03 07:02:00,2,drinks
2019-09-03 07:03:00,3,dairy
2019-09-03 07:05:00,1,drinks
2019-09-03 07:06:00,4,dairy
...,...,...
2019-09-03 21:47:00,1421,drinks
2019-09-03 21:47:00,1422,dairy
2019-09-03 21:48:00,1421,checkout


In [5]:
monday[monday['customer_no'] == 1]

,customer_no,location
timestamp,,
2019-09-02 07:03:00,1,dairy
2019-09-02 07:05:00,1,checkout


In [6]:
#add day column and create one dataframe with all data
monday['day'] = 'monday'
tuesday['day'] = 'tuesday'
wednesday['day'] = 'wednesday'
thursday['day'] = 'thursday'
friday['day'] = 'friday'



df =  pd.concat([monday, tuesday, wednesday, thursday, friday])
df.sort_values('customer_no').head()

,customer_no,location,day
timestamp,,,
2019-09-02 07:03:00,1,dairy,monday
2019-09-05 07:02:00,1,checkout,thursday
2019-09-04 07:02:00,1,checkout,wednesday
2019-09-04 07:00:00,1,fruit,wednesday
2019-09-06 07:00:00,1,dairy,friday


In [7]:
#create customer id column for unique keys
df['customer_id']= df.day + '_' + df.customer_no.astype(str)

In [8]:
df

,customer_no,location,day,customer_id
timestamp,,,,
2019-09-02 07:03:00,1,dairy,monday,monday_1
2019-09-02 07:03:00,2,dairy,monday,monday_2
2019-09-02 07:04:00,3,dairy,monday,monday_3
2019-09-02 07:04:00,4,dairy,monday,monday_4
2019-09-02 07:04:00,5,spices,monday,monday_5
...,...,...,...,...
2019-09-06 21:50:00,1500,dairy,friday,friday_1500
2019-09-06 21:50:00,1507,checkout,friday,friday_1507
2019-09-06 21:50:00,1508,checkout,friday,friday_1508


In [9]:
df[df.location=='checkout']

,customer_no,location,day,customer_id
timestamp,,,,
2019-09-02 07:05:00,1,checkout,monday,monday_1
2019-09-02 07:05:00,5,checkout,monday,monday_5
2019-09-02 07:06:00,2,checkout,monday,monday_2
2019-09-02 07:06:00,3,checkout,monday,monday_3
2019-09-02 07:07:00,8,checkout,monday,monday_8
...,...,...,...,...
2019-09-06 21:47:00,1502,checkout,friday,friday_1502
2019-09-06 21:48:00,1501,checkout,friday,friday_1501
2019-09-06 21:48:00,1504,checkout,friday,friday_1504


In [10]:
#tasks : - we need to consider every minute (each minute one row!)
# --> groupby('customer_no').resample('1Min').ffill()
#       - not every customer left the supermarket (dairy diaries)  
# --> checkout time = 9.59pm    
#       - we need the transition info in the table (from where to where are cust going in probabilities)
# --> transition matrix : a. shift(1) the location column to get the "from" column
#                         b. fillna('checkout') [there is one NaN in the first row]
#.                        c. pd.crosstab(df['from'], df['location'], normalize='index') 
#[split it if you want certain windows of time]

In [11]:
# Get all customers
total_customers = set(df['customer_id']) # use set-method to get a list with customers numbers
print("Total customers: ", len(total_customers))

Total customers:  7445


In [12]:
# Display the number of customers at checkout over time
customers_checkout = set(df.loc[df['location'] == 'checkout', 'customer_id']) # use set-method to get a list with customers numbers
print("Checked out customers: ", len(customers_checkout))

Checked out customers:  7417


In [13]:
# Display the number of customers that didn't checkout
customers_no_checkout = total_customers.difference(customers_checkout) # user difference-method to get a list with the customers numbers
print("Not checked out customers: ", customers_no_checkout)

Not checked out customers:  {'thursday_1533', 'tuesday_1422', 'friday_1509', 'monday_1433', 'tuesday_1411', 'wednesday_1528', 'monday_1439', 'friday_1505', 'thursday_1532', 'monday_1443', 'friday_1510', 'wednesday_1520', 'monday_1446', 'monday_1445', 'monday_1440', 'monday_1430', 'monday_1447', 'wednesday_1527', 'friday_1503', 'wednesday_1530', 'friday_1494', 'friday_1496', 'wednesday_1529', 'friday_1506', 'monday_1441', 'friday_1500', 'thursday_1527', 'monday_1437'}


In [14]:
def add_checkout(df, customer_ids):
    for c_id in customer_ids:
        date= df[df['customer_id']== c_id].index.day[0]
        #c_no = df.loc[df['customer_id']== c_id, 'customer_no'].values[0]
        day = c_id.split("_")[0]
        c_no = c_id.split("_")[1]
        df_tmp = pd.DataFrame(data=[[c_no, 'checkout', day, c_id]], index=[pd.to_datetime(f'2019-09-{date} 21:59:00')], columns=['customer_no', 'location', 'day','customer_id'])
        df = pd.concat([df, df_tmp])
    return df    

In [15]:
len(df.customer_id.unique())

7445

In [16]:
df.sort_values('customer_id')

,customer_no,location,day,customer_id
timestamp,,,,
2019-09-06 07:00:00,1,dairy,friday,friday_1
2019-09-06 07:04:00,1,spices,friday,friday_1
2019-09-06 07:05:00,1,checkout,friday,friday_1
2019-09-06 07:06:00,10,fruit,friday,friday_10
2019-09-06 07:11:00,10,checkout,friday,friday_10
...,...,...,...,...
2019-09-04 16:59:00,998,checkout,wednesday,wednesday_998
2019-09-04 16:53:00,998,dairy,wednesday,wednesday_998
2019-09-04 16:57:00,998,fruit,wednesday,wednesday_998


In [17]:
df[df.location=='checkout']

,customer_no,location,day,customer_id
timestamp,,,,
2019-09-02 07:05:00,1,checkout,monday,monday_1
2019-09-02 07:05:00,5,checkout,monday,monday_5
2019-09-02 07:06:00,2,checkout,monday,monday_2
2019-09-02 07:06:00,3,checkout,monday,monday_3
2019-09-02 07:07:00,8,checkout,monday,monday_8
...,...,...,...,...
2019-09-06 21:47:00,1502,checkout,friday,friday_1502
2019-09-06 21:48:00,1501,checkout,friday,friday_1501
2019-09-06 21:48:00,1504,checkout,friday,friday_1504


In [18]:
df

,customer_no,location,day,customer_id
timestamp,,,,
2019-09-02 07:03:00,1,dairy,monday,monday_1
2019-09-02 07:03:00,2,dairy,monday,monday_2
2019-09-02 07:04:00,3,dairy,monday,monday_3
2019-09-02 07:04:00,4,dairy,monday,monday_4
2019-09-02 07:04:00,5,spices,monday,monday_5
...,...,...,...,...
2019-09-06 21:50:00,1500,dairy,friday,friday_1500
2019-09-06 21:50:00,1507,checkout,friday,friday_1507
2019-09-06 21:50:00,1508,checkout,friday,friday_1508


In [19]:
filled_data = []

checkout_customers = set(df.loc[df['location'] == 'checkout', 'customer_id'])
no_checkout_customers = list(total_customers.difference(checkout_customers))

#for i in enumerate(df.customer_id):
#checkout = 
df = add_checkout(df, no_checkout_customers)
df.tail()
#filled_data.append(checkout)

,customer_no,location,day,customer_id
2019-09-06 21:59:00,1506,checkout,friday,friday_1506
2019-09-02 21:59:00,1441,checkout,monday,monday_1441
2019-09-06 21:59:00,1500,checkout,friday,friday_1500
2019-09-05 21:59:00,1527,checkout,thursday,thursday_1527
2019-09-02 21:59:00,1437,checkout,monday,monday_1437


In [20]:
# Get all customers
total_customers = set(df['customer_id']) # use set-method to get a list with customers numbers
print("Total customers: ", len(total_customers))
# Display the number of customers at checkout over time
customers_checkout = set(df.loc[df['location'] == 'checkout', 'customer_id']) # use set-method to get a list with customers numbers
print("Checked out customers: ", len(customers_checkout))
customers_no_checkout = total_customers.difference(customers_checkout) # user difference-method to get a list with the customers numbers
print("Not checked out customers: ", customers_no_checkout)

Total customers:  7445
Checked out customers:  7445
Not checked out customers:  set()


In [21]:
df

,customer_no,location,day,customer_id
2019-09-02 07:03:00,1,dairy,monday,monday_1
2019-09-02 07:03:00,2,dairy,monday,monday_2
2019-09-02 07:04:00,3,dairy,monday,monday_3
2019-09-02 07:04:00,4,dairy,monday,monday_4
2019-09-02 07:04:00,5,spices,monday,monday_5
...,...,...,...,...
2019-09-06 21:59:00,1506,checkout,friday,friday_1506
2019-09-02 21:59:00,1441,checkout,monday,monday_1441
2019-09-06 21:59:00,1500,checkout,friday,friday_1500
2019-09-05 21:59:00,1527,checkout,thursday,thursday_1527


In [22]:
set(df.loc[df['location'] == 'checkout', 'customer_id'])


{'wednesday_388',
 'friday_1322',
 'friday_146',
 'thursday_692',
 'monday_171',
 'wednesday_394',
 'wednesday_1209',
 'friday_1443',
 'thursday_1218',
 'monday_684',
 'monday_415',
 'monday_587',
 'monday_999',
 'friday_1364',
 'wednesday_1020',
 'wednesday_593',
 'thursday_997',
 'wednesday_19',
 'wednesday_632',
 'friday_1340',
 'wednesday_1187',
 'thursday_1519',
 'friday_1306',
 'friday_761',
 'thursday_425',
 'wednesday_45',
 'thursday_426',
 'thursday_843',
 'tuesday_1204',
 'thursday_605',
 'tuesday_714',
 'wednesday_749',
 'friday_359',
 'thursday_620',
 'friday_121',
 'monday_1345',
 'friday_48',
 'wednesday_1144',
 'wednesday_716',
 'friday_593',
 'thursday_430',
 'monday_776',
 'monday_1039',
 'wednesday_1230',
 'tuesday_433',
 'wednesday_1222',
 'wednesday_805',
 'friday_1256',
 'tuesday_871',
 'tuesday_1138',
 'wednesday_243',
 'tuesday_187',
 'friday_792',
 'tuesday_336',
 'tuesday_278',
 'tuesday_872',
 'friday_46',
 'friday_130',
 'friday_697',
 'wednesday_704',
 'tues

In [23]:
for i, day in enumerate(df.day):
    print(add_checkout(day, no_checkout_customers[i], date[i], weekdays[i]))

NameError: name 'date' is not defined

In [ ]:
df

In [ ]:
# df = df.groupby('customer_no').resample('1Min').ffill()
# df.head()

In [ ]:
# IF we want to reset index
#df.reset_index(level='timestamp', inplace= True)
#df.drop('customer_no', axis=1, inplace=True)


In [ ]:
#for day in days ?

In [ ]:
#add 'from' column and replace 
test['from'].replace({'checkout':'entry'}, inplace = True)

In [ ]:
pd.crosstab(test('from'), test('location'), normalize='index')